# <center> Tâche 1 de la pipeline : transcription des audios à l'écrit</center>

A partir de nos données audios (générées dans le notebook *Dataset_Creation*), nous allons les transcrire à l'écrit en utilisant un modèle de *speech to text*. Le **défi** ici est de parvenir à transcrire les audios en conservant les erreurs afin de permettre à l'utilisateur de visualiser ses erreurs dans le texte, et par la suite le texte corrigé. Le problème est que la plupart des modèles de *speech to text* possèdent une couche de traitement de la langue leur permettant de corriger les fautes. Un **autre défi** auquel nous allons faire face est l'affinnage du modèle, pour tenter d'augmenter ses performances en particulier sur l'accent indien.

On commence par créer notre jeu de données comportant les audios précédents et leur transcriptions associées. Le but ici est de créer un jeu de données qui servira pour l'affinnage de notre STT.

In [2]:
pip install gTTS datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [3]:
pip install import_ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.7 MB/s eta 0:00:00


In [4]:
pip install jiwer # pour utiliser WER metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.7 MB/s eta 0:00:00


In [5]:
import pandas as pd
import os
import torchaudio
import torch
from datasets import load_dataset, concatenate_datasets, Dataset
from datasets.features.audio import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from gtts import gTTS
import import_ipynb
from Dataset_Creation import dataset_concat, nb_audio, audio_folder # import de variables provenant de l'autre notebook

importing Jupyter notebook from Dataset_Creation.ipynb


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1503 [00:00<?, ? examples/s]

Les retranscriptions audios se trouvent dans le dossier 'audio'. 
Nombre total d'audio créé: 1141 sur 1501 textes du jeu de données JFLEG


In [6]:
# CREATION METADATA (doc hugging face :https://huggingface.co/docs/datasets/audio_load)

metadata_csv_path = os.path.join(audio_folder, "metadata.csv")

if not os.path.exists(metadata_csv_path):
  # Liste des chemins des fichiers audio
  audio_files = [f"{i}.mp3" for i in range(0, nb_audio)]
  # Transcriptions associées
  transcriptions = dataset_concat["sentence"][:nb_audio]
  # Créer un DataFrame avec les noms de fichiers et les transcriptions
  df = pd.DataFrame({"file_name": audio_files, "transcription": transcriptions})
  # Enregistrer le DataFrame dans un fichier CSV
  df.to_csv(metadata_csv_path, index=False)
  print(f"Metadata CSV enregistré à : {metadata_csv_path}")
else:
  print("Le fichier metadata.csv existe déjà")

# CREATION DU LOAD_DATASET
dataset = load_dataset("audiofolder", data_dir=audio_folder, split="train")

Le fichier metadata.csv existe déjà


Resolving data files:   0%|          | 0/1142 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 1141
})

In [ ]:
dataset['audio']

[{'path': '/content/audio/0.mp3',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -6.14001419e-06, -6.29437864e-06, -6.43266594e-06]),
  'sampling_rate': 16000},
 {'path': '/content/audio/1.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': '/content/audio/10.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': '/content/audio/100.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': '/content/audio/1000.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': '/content/audio/1001.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': '/content/audio/1002.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': '/content/audio/1003.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': '/conten

## Prétraitement des données

1. Rééchantillonnage

Nos données avaient initialement un taux d'échantillonnage (*sampling_rate*) de 24KHz, or le modèle STT d'Hugging face a besoin d'un taux de 16KHz (doc : https://huggingface.co/facebook/wav2vec2-base-960h). On a donc fait un rééchantillonnage.

In [ ]:
# # Liste des chemins des fichiers audio
# audio_files = [os.path.join(audio_folder, f"{i}.mp3") for i in range(0, nb_audio)]

# # Rééchantillonnage des fichiers audio à 16000 Hz
# target_sampling_rate = 16000
# resample_transform = torchaudio.transforms.Resample(orig_freq=24000, new_freq=target_sampling_rate)

# # Rééchantillonner et sauvegarder chaque fichier audio
# for audio_file in audio_files:
#     # Charger l'audio
#     waveform, original_sampling_rate = torchaudio.load(audio_file, normalize=True)

#     # Rééchantillonner l'audio
#     resampled_waveform = resample_transform(waveform)

#     # Sauvegarder l'audio rééchantillonné dans un nouveau fichier
#     new_audio_file = os.path.join(audio_folder, os.path.basename(audio_file))
#     torchaudio.save(new_audio_file, resampled_waveform, target_sampling_rate)

#     # print(f"Audio rééchantillonné enregistré : {new_audio_file}")


Nos données sont maintenant bien échantillonées à 16KHz.

2. Division en jeu d'entrainement et de test

In [8]:
from datasets import DatasetDict

# Diviser l'ensemble de données en portions "train" et "test"
train_percentage = 0.8
train_size = int(len(dataset) * train_percentage)

# Créer un DatasetDict
dataset_dict = DatasetDict({
    "train": dataset.select(range(train_size)),
    "test": dataset.select(range(train_size, len(dataset)))
})

# Charger la portion "train"
train_dataset = dataset_dict["train"]

# Charger la portion "test"
test_dataset = dataset_dict["test"]

print(train_dataset)
print(test_dataset)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 912
})
Dataset({
    features: ['audio', 'transcription'],
    num_rows: 229
})


## Choix des modèles

On va maintenant comparer deux modèles de ASR (*automatic speech recognition*) : l'un comportant une couche de traitement de la langue, et l'autre non. On veut trouver un modèle pour cette tâche qui ne corrige pas déjà les fautes de grammaire qui sont audibles dans les audios. Ces deux modèles sont les suivants :
- **Wav2Vec2** (uniquement basé sur la phonétique)
- **Whisper** (possédant une couche de traitement de la langue)

Testons les deux modèles sur un exemple de notre dataset pour avoir un apperçu de leur comportement :

Test du modèle Wav2Vec2 :

In [ ]:
from IPython.display import Audio
from transformers import pipeline


sample = dataset[4] # échantillon pour tester le modèle

pipe_wav2vec2 = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-100h")
prediction_wav2vec2 =  pipe_wav2vec2(sample["audio"].copy())["text"]
reference = sample["transcription"]

print("\nPrediction à partir de l'audio :", prediction_wav2vec2)
print("Phrase réelle associée à l'audio :", reference)
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

Some weights of the model checkpoint at facebook/wav2vec2-base-100h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.mask_time_emb_vector', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-100h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav


Prediction à partir de l'audio : I DON'T HAVE A GARB BUT I DREAMING IT OFF
Phrase réelle associée à l'audio : I dont have a car but i dreaming it off .


Test du modèle Whisper :

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe_whisper = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device
)
prediction_whisper = pipe_whisper(sample["audio"].copy(), max_new_tokens=256)["text"]

print("\nPrediction à partir de l'audio :", prediction_whisper)
print("Phrase réelle associée à l'audio :", reference)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



Prediction à partir de l'audio :  I don't have a car but I dreaming it off.
Phrase réelle associée à l'audio : I dont have a car but i dreaming it off .


Il semble que le modèle Whisper fonctionne mieux que le Wav2Vec2. En effet, le traitement de la langue présent dans Whisper permet de n'obtenir en sortie que des mots qui existent dans le vocabulaire anglais, contrairement à Wav2Vec2 qui ne fonctionne qu'à la phonétique et qui donc retranscrit des mots inexistants. De plus, Whisper semble tout de même retranscrire les erreurs grammaticales comme sur l'exemple ci-dessus. On va calculer le WER de chacun des deux modèles sur nos données pour confirmer cette hypothèse.

A savoir : au plus le WER est faible au plus notre modèle à réussi à retranscrire les bon mots + possible normalisation (à voir) + possible d'utiliser le CER

Pour régler le soucis du WER qui pénalise fortement les fautes d'orthographe on pourra utiliser le CER (character au lieu de word)

In [ ]:
cer_metric = load_metric("cer")

cer_wav2vec2 = cer_metric.compute(references=[reference], predictions=[prediction_wav2vec2])
print(cer_wav2vec2)

cer_whisper = cer_metric.compute(references=[reference], predictions=[prediction_whisper])
print(cer_whisper)

0.7981651376146789
0.07339449541284404


On peut également normaliser nos prédictions avant de calculer le WER (ou CER) pour ne pas compter les fautes de ponctuation (seul le modèle whisper est concerné car wav2vec2 n'est pas un modèle orthographique). En effet dans notre cas nous nous intéressons surtout à la bonne retranscription des mots et des fautes de grammaire, mais pas à la ponctuation. Aussi dans l'exemple précédent avec whisper on peut voir qu'un point a été ajouté avant le dernier mot de la phrase dans la prédiction ce qui a un impact sur le WER, or la normalisation permettra par exemple de supprimer ce point et la majuscule qui suit et ainsi obtenir une meilleure métrique.

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

prediction = prediction_whisper
normalized_prediction = normalizer(prediction)

print(prediction_whisper)
print(normalized_prediction)

 So I think we cannot live if old people could not find sciences and technologies and they did not. Developed
 so i think we cannot live if old people could not find sciences and technologies and they did not developed


In [ ]:
wer = wer_metric.compute(references=[reference], predictions=[normalized_prediction])
wer

0.36363636363636365

In [ ]:
print(reference)
print(prediction_whisper)
print(normalized_prediction)

So I think we can not live if old people could not find siences and tecnologies and they did not developped . 
 So I think we cannot live if old people could not find sciences and technologies and they did not. Developed
 so i think we cannot live if old people could not find sciences and technologies and they did not developed


## Evaluation des modèles ASR sur les données de test

In [9]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

def speech_recognition_inference(dataset, pipe, batch_size):
  all_predictions = []

  for prediction in tqdm(pipe(
          KeyDataset(dataset, "audio"),
          max_new_tokens=256,
          generate_kwargs={"task": "transcribe"},
          batch_size=batch_size,
      ),
      total=len(dataset),
  ):
      all_predictions.append(prediction["text"])
      del prediction # release useless variables from memory (to have more space)

  return all_predictions

Idée pour wav2vec2 : utiliser une autre version du modèle plus petite, ou supprimer les variables inutiles au fur et à mesure `del prediction  # Explicitly release memory`)

In [10]:
from transformers import pipeline

pipe_wav2vec2 = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-100h")
predictions_wav2vec2 = speech_recognition_inference(test_dataset, pipe_wav2vec2, 8)

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-100h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.mask_time_emb_vector', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-100h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

100%|██████████| 229/229 [12:11<00:00,  3.19s/it]


In [ ]:
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe_whisper = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)
predictions_whisper = speech_recognition_inference(test_dataset, pipe_whisper, 32)

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

100%|██████████| 229/229 [00:45<00:00,  5.00it/s]


In [11]:
predictions_wav2vec2

['I CASET DOUT THAT TWENTY YEARS IS TO SHORT TIME TO PROVE THE PROBLEM',
 'SO FOR THESE REASONS I REALLY BELIEVE THAT IT ES IMPORTANT TO HAVE A BROAD KNOWLEDGE ON DIFFERENT SUBJECTS INSTEAD OF BEING FOLKESD ON JUST ONE BECAUSE SOME TIME THE ANSWER THAT WE ARE SEARCHING FOR WE CAN EASILY FIND IT IN A FIELD THAT CAN SEEMS NOT RELATED TO OUR PURPOSE',
 'I AM PLEASED TO KNOW INTELLIGENT BEPLY AND LEARN ABOUT THINGS WHICH I DO UN KNOW',
 'TO RESOLVE THIS PROBLEM THE GOVERNMENTS OF THE CITIES ARE LIKELY TO IMPROVE THE PUBLIC TRANSPORTATION SYSTEM BY BUILDING NEW SUBBWAY SYSTEM AND ADDING MO LINES FOR BUSSELS',
 'I KNOW LOTS OF OLD PEOPLE WHO GO TO PARTIES STAY ALWAYS IN CONTACT WITH OTHERS PEOPLE',
 "BUT THESE THING SEEM TO HARD FOR OLD PEOPLE THEY MOVE SO HARDLY AND DO ON'T HAVE HEALTH BODY TO PLAY THIS THING LIKE SPORTS",
 'FOR EXAMPLE IF YOU UNDERSTAND THE IDEA THAT ALL DISEASES CAUSED BY MY CRUORGANISMS ARE TREATED WITH THE ANTY BIORTICS YOU WILL NEVER USE ANTY DEPRESENCE',
 "IT FIGURES 

In [14]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from datasets import load_metric

def compute_wer(references, predictions, isNormalisation):
  wer_metric = load_metric("wer")
  if isNormalisation is False:
    wer = 100 * wer_metric.compute(references=references, predictions=predictions)
  else :
    normalizer = BasicTextNormalizer()
    predictions_norm = [normalizer(pred) for pred in predictions]
    references_norm = [normalizer(ref) for ref in references]
    wer = 100 * wer_metric.compute(references=references_norm, predictions=predictions_norm)

  return wer

In [16]:
wer_ortho_wav2vec2 = compute_wer(test_dataset["transcription"], predictions_wav2vec2, False)
wer_norm_wav2vec2 = compute_wer(test_dataset["transcription"], predictions_wav2vec2, True)
print("Modèle Wav2Vec2 :")
print("WER avec normalisation:", wer_norm_wav2vec2)

Modèle Wav2Vec2 :
WER : 19.095849802371543


Le modèle Wav2Vec2 n'est pas un modèle orthographique, c'est-à-dire qu'il ne fait pas de transcription avec de la ponctuation. De plus sa sortie est tout en majuscule. On est donc obligé de normaliser les références et les inférences pour pouvoir calculer la WER (car les références contiennent de la ponctuation).

In [ ]:
wer_ortho = compute_wer(test_dataset["transcription"], predictions_whisper, False)
wer_norm = compute_wer(test_dataset["transcription"], predictions_whisper, True)
print("Modèle Whisper :")
print("WER sans normalisation :", wer_ortho)
print("WER avec normalisation :", wer_norm)

WER sans normalisation : 28.966914247130315
WER avec normalisation : 8.473320158102766


Finalement, notre hypothèse comme quoi le modèle Whisper est meilleur dans notre cas que le modèle Wav2Vec2 est validée. On observe que la WER de Wav2Vec2 est de 19% et celle de Whisper, après normalisation, de 8.5%. Cela signifie que le modèle Whisper est parvenue à retranscrire les audios de manière correcte, tout en prenant en compte les erreurs grammaticales qui y sont présentes.

On va donc maintenant essayer d'améliorer cette métrique en faisant un affinage de notre modèle Whisper.

## Fine tunning du modèle Whisper

Nous allons faire un fine tunning sur notre modèle pour améliorer ses performances, en particulier sur nos audios qui sont des audios de personne parlant anglais avec l'accent indien.